# research_code

> A selection of useful code chunks for analysing THz-TDS data.

In [1]:
#| hide
from fastcore.test import *
import fastcore.docments as docment
from nbdev.showdoc import DocmentTbl

In [2]:
#| default_exp research_code

In [3]:
#| export 
import numpy as np
from scipy import constants as consts
from scipy.fft import fft,fftfreq,rfft, fftshift,fftn
import pandas as pd

In [4]:
#| export
def exp_decay(x: np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Pre-factor of the exponential decay. 
    tau: np.single, # Decay constant.
    x0: np.single, # Offset for the independent variable .
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Single exponentially decaying function.
    '''
    decayOutput = Amplitude * np.exp(-(x-x0)/tau)
    
    return decayOutput

In [5]:
#| export
def biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1: np.single, # Prefactor of the first exponential decay.
    tau1: np.single, # Decay constant of the first exponential term.
    Amplitude2: np.single, # Prefactor of the second exponential decay.
    tau2: np.single, # Decay constant of the second exponential term.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function.
    '''
    decayOutput = Amplitude1 * np.exp(-(x-x0)/tau1) + Amplitude2 * np.exp(-(x-x0)/tau2)
    
    return decayOutput

In [6]:
#| export
def gaussian(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the gaussian.
    sigma: np.single, # Standard deviation of the gaussian.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the gaussian function.
    '''
    Gaussian function.
    '''
    outputGaussian = Amplitude * np.exp(-((x-x0)/sigma)**2)

    return outputGaussian

In [7]:
#| export
def convolved_exp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the exponential decay.
    tau: np.single, # Decay constant.
    sigma: np.single, # Standard deviation of the gaussian
    x0: np.single, # Offset for the indpendent variable.
    y0: np.single, # Offset in y.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Single exponentially decaying function convolved with a gaussian.
    '''
    L = len(x) # length of the independent variable.

    exp_term = exp_decay(x,Amplitude,tau,x0) # The exponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved = np.convolve(exp_term,gaussian_term,mode='full')[0:L] # convolved function.

    convolved *= (Amplitude-y0)/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

In [8]:
#| export
def convolved_biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1:np.single, # Prefactor of the first exponential decay.
    tau1:np.single, # Decay constant of the first exponential term.
    Amplitude2:np.single, # Prefactor of the second exponential decay.
    tau2:np.single, # Decay constant of the second exponential term.
    sigma:np.single, # Standard deviation of the gaussian
    x0:np.single, # Offset for the indpendent variable.
    y0:np.single, # Offset in y.
    ) ->np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function convolved with a gaussian.
    '''    
    L = len(x) # length of the independent variable.

    biexp_term = biexp_decay(x,Amplitude1,tau1,Amplitude2,tau2,x0) # The biexponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved = np.convolve(biexp_term,gaussian_term,mode='full')[0:L] # convolved function.

    convolved *= (Amplitude1+Amplitude2-y0)/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

## Functions for use with un-excited THz-TDS data

In [9]:
#| export
def data_reformatter(df):

    """
    The purpose of this function is to repackage or change the format of the data, 
    such that rather than having two columns with a large number of rows, we have
    different columns for each scan. The data is also augmented with two additional 
    rows - the mean and the std of the scans.

    """
    data = df.to_numpy()

    Ntot,_ = np.shape(data) # totoal number of rows

    delays = np.unique(data[:,0]) # unique delay positions in mm
    
    delays = 2*delays/(consts.c) * 1e-3 * 1e12
    
    N = len(delays) # number of delays per scan

    M = int(Ntot/N) # number of scans

    df2 = pd.DataFrame(np.reshape(data[:,1],(M,N)).T) # reshape the data and put it into a dataframe

    col_names = {i:f'scan{i}' for i in range(M)} # create an array with names like 'scan0, scan1, ....'
    
    df2 = df2.rename(columns=col_names) # rename the columns

    mean_col = df2.mean(axis=1) # calculate a new column containing the mean value for each delay

    std_col = df2.std(axis=1) # calculate a new column containing the std value for each delay

    df2['mean'] = mean_col # augment the dataframe with a new column containing the mean

    df2['std'] = std_col # augment the dataframe with a new column containing the std
    
    df2.insert(loc=0,column='delay',value=delays) # add a new column at the start of the DataFrame containing the delays

    return df2

In [10]:
#| export
def fft_simple(df,**kwargs):
    """
    This is a simple implementation of fft that will work for a DataFrame object
    """
    # temporal resolution
    dt = np.abs(df['delay'][1] - df['delay'][0])
    
    # sampling rate
    S = 1/dt

    pad = kwargs['pad']

    N_old,M = np.shape(df)

    print(N_old,M)

    scan_cols = [s for s in df_g if s.startswith('scan')]
    
    M = len(scan_cols)
    
    scans = df[scan_cols].to_numpy()

    zero_pad = np.zeros((pad,M))

    data_pad = np.vstack((scans,zero_pad)) # just stacking at end, should probably zero pad both sides

    ##t_pad = np.array([df['delay'][0] + dt*i for i in range(N_old+pad)])

    freqs = fftfreq(N_old+pad,dt)
    
    # FFT of signal
    N_freq = len(freqs)
    
    N2 = int(np.floor(N_freq/2))

    signal = fftn(data_pad,axes=0)

    PSD = np.zeros(np.shape(signal))
    
    for i in range(np.size(signal,1)):
        PSD[:,i] = np.abs(signal[:,i])**2

    df2 = pd.DataFrame(signal[1:N2])

    col_names = {i:f'scan{i}' for i in range(M)}

    cols = ['scan'+str(i) for i in range(M)]
    
    df2.rename(columns=col_names,inplace=True)

    mean_col = df2.mean(axis=1)

    std_col = np.std(df2.to_numpy(),axis=1)

    df2.insert(loc=0,column='freq',value=freqs[1:N2])

    df2['mean'] = mean_col

    df2['std'] = std_col

    df2['PSD'] = np.abs(df2['mean'])**2

    df2['PSD_err'] = np.std(np.abs(df2[cols].to_numpy())**2,axis=1)
    
    return df2

In [11]:
#| export
def transmission(df1,df2):

    cols = [s for s in df_g if s.startswith('scan')]
    
    M = len(cols)

    col_names = {i:'scan'+str(i) for i in range(M)}
    
    df_T = df1[cols].to_numpy()

    ref_mean = df2['mean'].to_numpy()

    dnew = np.zeros((np.size(df_T,0),M),dtype=np.complex128)
    
    for i in range(M):

        temp = df_T[:,i]/ref_mean
        
        tmod = np.abs(df_T[:,i]/ref_mean)
                
        dnew[:,i] = temp

    T_mean = np.mean(dnew,axis=1)
    
    T_std = np.std(dnew,axis=1)
    
    df = pd.DataFrame(dnew)

    df.rename(columns=col_names,inplace=True)

    df['T'] = T_mean

    df['T_err'] = T_std
    
    df.insert(loc=0,column='freq',value=df1['freq'])

    return df

In [12]:
#| export
def conductivity(T_in,**kwargs):
    """
    This function calculates the complex conductivity according to equation 1 or 2
    """
    n_substrate = 3.45+0.25j # Assuming a constant, real refractive index for the substrate. This is not true, but need to dig up the measured data.
    Z_0 = 377 # ohms, vacuum permittivity
    n_A = n_substrate + 1 # from equation section
    n_B = n_substrate + 1 # from equation section
    e2_h = consts.e**2/consts.h # factor that converts to units of e^2/h
    
    cols = [s for s in df_g if s.startswith('scan')]
    
    M = len(cols)

    col_names = {i:'scan'+str(i) for i in range(M)}
    
    data_in = T_in[cols].to_numpy()

    dnew = np.zeros((np.size(data_in,0),M),dtype=np.complex128)

    if (kwargs["equation"]=="eq1"):
        
        for i in range(M):
            
            dnew[:,i] = (1/e2_h) * n_A/Z_0 * ((1/data_in[:,i]) - 1)

    if (kwargs["equation"]=="eq2"):

        for i in range(M):
            
            dnew[:,i] = (1/e2_h) * (n_A * np.sqrt(n_A**2 + 4*n_B*(n_A+n_B)*data_in[:,i]) - \
            n_A**2 - 2*n_A*n_B*data_in[:,i])/(2*n_B*Z_0*data_in[:,i])
            
    sigma_mean = np.mean(dnew,axis=1)
    
    sigma_std = np.std(dnew,axis=1)
    
    df = pd.DataFrame(dnew)

    df.rename(columns=col_names,inplace=True)

    df['mean'] = sigma_mean

    df['std'] = sigma_std

    df['real'] = np.mean(np.real(dnew),axis=1)

    df['real_err'] = np.std(np.real(dnew),axis=1)

    df['imag'] = np.mean(np.imag(dnew),axis=1)

    df['imag_err'] = np.std(np.imag(dnew),axis=1)

    return df

In [13]:
#| hide
from nbdev.showdoc import *

In [14]:
#| export
def foo(): pass

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()